make pipeline

In [7]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sksurv.ensemble import RandomSurvivalForest, GradientBoostingSurvivalAnalysis
from sksurv.svm import FastKernelSurvivalSVM
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.linear_model import CoxnetSurvivalAnalysis
from sklearn.pipeline import  make_pipeline
from sksurv.metrics import concordance_index_censored,concordance_index_ipcw
from sklearn.preprocessing import StandardScaler

In [3]:
training_df = pd.read_csv("../../data/train_test/training.csv")

In [4]:
X_train = training_df.drop(columns=['survival_status', 'survival_time', 'recurrence', 'metastasis'])
y_train_struct = training_df[['survival_status', 'survival_time']].to_records(index=False).astype([('Status', 'bool'), ('Survival', 'float64')])

In [8]:
pipelines = {
    'coxnet': make_pipeline(StandardScaler(), CoxnetSurvivalAnalysis()),
    'rf': make_pipeline(StandardScaler(), RandomSurvivalForest()),
    'gb': make_pipeline(StandardScaler(), GradientBoostingSurvivalAnalysis()),
    'fksvm': make_pipeline(StandardScaler(), FastKernelSurvivalSVM())
}


In [9]:
models = dict()

for model, pipeline in pipelines.items():
    print(f"{model}: Fitting...")
    models[model] = pipeline.fit(X_train, y_train_struct)
    print("Done.")
    print("="*8)

coxnet: Fitting...
Done.
rf: Fitting...
Done.
gb: Fitting...
Done.
fksvm: Fitting...
Done.


/home/azetry/Projects/CBMP_FinalProject/env/lib/python3.10/site-packages/sklearn/pipeline.py:382: ConvergenceWarning: Optimization did not converge: Warning: Maximum number of iterations has been exceeded.
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


In [12]:
for name, model in models.items():
    print(f"{name}: c-index")
    print(concordance_index_censored(y_train_struct['Status'],y_train_struct['Survival'],model.predict(X_train))[0])

coxnet: c-index
1.0
rf: c-index
0.9452279884978776
gb: c-index
0.9942489387922772
fksvm: c-index
0.9990414897987129
